## Zadanie domowe - Algorytm Canny'ego

Celem zadania domowego jest wykonanie pełnej implementacji algorytmu Canny'ego.

W ramach ćwiczenia w trakcie laboratorium wyznaczono obrazy $g_{NH}$ i $g_{NL}$.
Dla przypomnienia:
Można powiedzieć, że na obrazie $g_{NH}$ są "pewne" krawędzie.
Natomiast na $g_{NL}$ "potencjalne".
Często krawędzie "pewne" nie są ciągłe.
Wykorzystuje się więc krawędzie "potencjalne", aby uzupełnić nieciągłości.
Procedura wygląda następująco:
1. Stwórz stos zawierający wszystkie piksele zaznaczone na obrazie $g_{NH}$.
W tym celu wykorzystaj listę współrzędnych `[row, col]`.
Do pobrania elementu z początku służy metoda `list.pop()`.
Do dodania elementu na koniec listy służy metoda `list.append(new)`.
2. Stwórz obraz, który będzie zawierał informację czy dany piksel został już odwiedzony.
3. Stwórz obraz, który zawierać będzie wynikowe krawędzie.
Jej rozmiar jest równy rozmiarowi obrazu.
4. Wykonaj pętlę, która będzie pobierać elementy z listy, dopóki ta nie będzie pusta.
W tym celu najlepiej sprawdzi się pętla `while`.
    - W każdej iteracji pobierz element ze stosu.
    - Sprawdź, czy dany element został już odwiedzony.
    - Jeśli nie został, to:
        - Oznacz go jako odwiedzony,
        - Oznacz piksel jako krawędź w wyniku,
        - Sprawdź otoczenie piksela w obrazie $g_{NL}$,
        - Dodaj do stosu współrzędne otoczenia, które zawierają krawędź.
        Można to wykonać np. pętlą po stworzonym otoczeniu.
7. Wyświetl obraz oryginalny, obraz $g_{NH}$ oraz obraz wynikowy.
8. Porównaj wynik algorytmu z wynikiem OpenCV.

Pomocnicze obrazy $g_{NH}$ i $g_{NL}$ zostały wprowadzone dla uproszczenia opisu.
Algorytm można zaimplementować w bardziej "zwarty" sposób.

Na podstawie powyższego opisu zaimplementuj pełny algorytm Canny'ego.

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os

if not os.path.exists("dom.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/09_Canny/dom.png --no-check-certificate

In [ ]:
def my_canny(image, t_low, t_high):

    image_filtered = cv2.GaussianBlur(image, (5, 5), 0)
    
    Sobel1 = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    Sobel2 = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])
    
    Gx = cv2.filter2D(image_filtered, -1, Sobel1)
    Gy = cv2.filter2D(image_filtered, -1, Sobel2)
    
    M = np.sqrt(Gx**2+Gy**2)
    
    alpha_rad = np.arctan2(Gy, Gx)
    alpha_deg = alpha_rad*180/np.pi
    alpha_deg[alpha_deg < 0] += 180
    
    directions = np.zeros(shape=image.shape)

    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if (0 <= alpha_deg[i,j] < 22.5) or (157.5 <= alpha_deg[i,j] <= 180):
                directions[i, j] = 0
            elif (22.5 <= alpha_deg[i,j] < 67.5):
                directions[i, j] = 45
            elif (67.5 <= alpha_deg[i,j] < 112.5):
                directions[i, j] = 90
            elif (112.5 <= alpha_deg[i,j] < 157.5):
                directions[i, j] = 135
    
    gN = nonmax(directions, M)
    gNh = gN >= t_high
    gNl = np.where(np.logical_and(t_high > gN, gN >= t_low), 1, 0)

    stack = []
    for i in range(1, image.shape[0]-1):
        for j in range(1, image.shape[1]-1):
            if gNh[i, j] == 1:
                stack.append([i, j])
    
    visited = np.zeros(shape=image.shape)
    result_edges = np.zeros(shape=image.shape)
    while stack:
        i, j = stack.pop()
        if visited[i, j] == 0:
            visited[i, j] = 1
            result_edges[i, j] = 1
            for x in range(i-1, i+2):
                for y in range(j-1, j+2):
                    if gNl[x, y] == 1:
                        stack.append([x, y])
    return gNl, gNh, result_edges

In [ ]:
def nonmax(directions, M):
    gN = np.zeros(shape=directions.shape)
    for i in range(1, directions.shape[0]-1):
        for j in range(1, directions.shape[1]-1):
            if(directions[i, j] == 0):
                if(M[i, j-1] > M[i, j] or M[i, j+1] > M[i, j]):
                    gN[i, j] = 0
                else:
                    gN[i, j] = M[i, j]
            elif(directions[i, j] == 45):
                if(M[i+1, j+1] > M[i, j] or M[i-1, j-1] > M[i, j]):
                    gN[i, j] = 0
                else:
                    gN[i, j] = M[i, j]
            elif(directions[i, j] == 90):
                if(M[i-1, j] > M[i, j] or M[i+1, j] > M[i, j]):
                    gN[i, j] = 0
                else:
                    gN[i, j] = M[i, j]
            elif(directions[i, j] == 135):
                if(M[i+1, j-1] > M[i, j] or M[i-1, j+1] > M[i, j]):
                    gN[i, j] = 0
                else:
                    gN[i, j] = M[i, j]
    return gN

In [ ]:
house = cv2.imread('dom.png', cv2.IMREAD_GRAYSCALE)
gNl, gNh, result = my_canny(house, 5, 10)

plt.figure(figsize=(10, 8))
ax = plt.subplot(2, 2, 1)
ax.imshow(house, 'gray')
plt.xticks([]), plt.yticks([])
plt.title('Oryginalny obraz')

ax = plt.subplot(2, 2, 2)
ax.imshow(gNh, 'gray', vmin=0, vmax=1)
plt.xticks([]), plt.yticks([])
plt.title('Obraz gNh')

ax = plt.subplot(2, 2, 3)
ax.imshow(result, 'gray', vmin=0, vmax=1)
plt.xticks([]), plt.yticks([])
plt.title('Wynik pełnego algorytmu Cannego')

ax = plt.subplot(2, 2, 4)
ax.imshow(cv2.Canny(house, 2, 6, None, 3, 1), 'gray')
plt.xticks([]), plt.yticks([])
plt.title('Funkcja z biblioteki')

plt.show()